In [217]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [218]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def printValues(self):
        print("\n")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

# Please fill in all inputs and their units below
# v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v 

Units Set: ksi, kip, in, in2

In [219]:
YoungsModulus = 30000
YM_Unit = "ksi"

Area = 1
A_Unit = "in2"

#is it symmetric? if so, about what element?
Symmetric = True
SymmetryAbout = 11

P = 20
Force_Unit = "kip"

# node connections for each element
nArray = np.array([[1,2], [1,3], [2,3], [2,4], [3,4], [3,5], [4,5], [4,6], [5,6], [5,7], [6,7]])

# list of lengths corresponding to each element
# the length of element n is contained in lengths
lArray = np.array([6.5, 6, 2.5, 6.5, 7.8102496759, 6, 5, 6.5, 9.6046863561, 6, 7.5])
lArray *= 12

Length_unit = "in"

# list of angles relative to positive x axis for each element
theta = m.atan(7.5/18)
aArray = np.array([theta, 0, -m.pi*0.5, theta, m.atan(5/6), 0, -m.pi*0.5, theta, m.atan(7.5/6), 0, -m.pi*0.5])

Angle_unit = "rad"

# input boundary conditions below
# for displacements: [node#, u, v] ->>> Default all u,v are unknown (None)
# for forces: [node#, Fx, Fy] ->>> Default all fx, fy are assumed no concentrated load (0)

uv_BCs = np.array([[3, 0, 0], [6, 0, None], [7, 0, None]])

force_BCs = np.array([[1, 0, -P], [5, 0, -P], [6, None, 0], [7, None, (-P/2)]])

if len(aArray) != len(lArray) or len(nArray) != len(lArray) or len(nArray) != len(aArray):
    print("Error! Inputs do not have the same amount of nodes or elements!")

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [220]:
E = Input("Young's Modulus (E)", YoungsModulus, YM_Unit)
E.printValues()

if Symmetric == False:
    A = Input("Area (A)", Area, A_Unit)
    A.printValues()

nodes = Input("Node numbers", nArray, "None")
nodes.printValues()

lengths = Input("Lengths", lArray, Length_unit)
lengths.printValues()

angles = Input("Element Angles (relative to the positive x axis)", aArray, Angle_unit)
angles.printValues()



name: Young's Modulus (E)
value: 30000
unit: ksi


name: Node numbers
value: [[1 2]
 [1 3]
 [2 3]
 [2 4]
 [3 4]
 [3 5]
 [4 5]
 [4 6]
 [5 6]
 [5 7]
 [6 7]]
unit: None


name: Lengths
value: [ 78.          72.          30.          78.          93.72299611
  72.          60.          78.         115.25623627  72.
  90.        ]
unit: in


name: Element Angles (relative to the positive x axis)
value: [ 0.39479112  0.         -1.57079633  0.39479112  0.69473828  0.
 -1.57079633  0.39479112  0.89605538  0.         -1.57079633]
unit: rad


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [221]:
numElems = len(nodes.value)
print(f"Number of elements: {numElems}")
numNodes = nodes.value.max()
print(f"Number of nodes: {numNodes}")

Number of elements: 11
Number of nodes: 7


Symmetry Code

In [222]:
if Symmetric == True:
    areas = np.full((numElems), Area, dtype=float)
    areas[SymmetryAbout - 1] = Area/2

    A = Input("Area (A)", areas, A_Unit)
    A.printValues()



name: Area (A)
value: [1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.5]
unit: in2


two reactions per node
<br>uvList -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.


In [223]:
uvList = [None]*(numNodes*2)
print("displacement bcs: [node number, x disp (u), y disp (v)]")
print(uv_BCs)

adjust_array(uvList, uv_BCs, "displacements")
print("u & v list:")
print(uvList)

displacement bcs: [node number, x disp (u), y disp (v)]
[[3 0 0]
 [6 0 None]
 [7 0 None]]
adjusting array to account for: displacements boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
case 4 for BC 2
case 1 for BC 3
case 4 for BC 3
done adjusting displacements array!
u & v list:
[None, None, None, None, 0, 0, None, None, None, None, 0, None, 0, None]


Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [224]:
#forceList = [0]*(numNodes*2)
forceList = np.zeros(((numNodes*2), 1))

nodesWithLoads = []

loadedNodes = 0

for i in range(len(force_BCs)):
    if (force_BCs[i,1]!= None and force_BCs[i,1] != 0) or (force_BCs[i,2] != None and force_BCs[i,2] != 0):
        loadedNodes += 1
        nodesWithLoads.append((force_BCs[i,0] - 1))


print(f"number of nodes with concentrated loads: {loadedNodes}")
print(f"at nodes: {nodesWithLoads} (Python adjusted nodes)")

#adjust force array given concentrated loading conditions
adjust_array(forceList, force_BCs, "forces")

print(f"force list: \n{forceList}")

number of nodes with concentrated loads: 3
at nodes: [0, 4, 6] (Python adjusted nodes)
adjusting array to account for: forces boundary conditions
case 3 for BC 1
Case 2 for BC 1
case 3 for BC 2
Case 2 for BC 2
case 1 for BC 3
case 4 for BC 3
case 1 for BC 4
Case 2 for BC 4
done adjusting forces array!
force list: 
[[  0.]
 [-20.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [-20.]
 [ nan]
 [  0.]
 [ nan]
 [-10.]]


Global [k] matrix is a symmetric array of shape (numNodes x 2, numNodes x 2)
<br>Bar Constants are one per element, stored in a numElemes * 1 array.

In [225]:
globalK = np.zeros(((numNodes*2),(numNodes*2)))
print(f"Global K array dimensions: {globalK.shape}")

barConstant = np.zeros((numElems,1))
for i in range(numElems):
    barConstant[i] = (E.value*A.value[i])/lengths.value[i]

print(f"Bar Constants = \n{barConstant}")

Global K array dimensions: (14, 14)
Bar Constants = 
[[ 384.61538462]
 [ 416.66666667]
 [1000.        ]
 [ 384.61538462]
 [ 320.09219983]
 [ 416.66666667]
 [ 500.        ]
 [ 384.61538462]
 [ 260.28960315]
 [ 416.66666667]
 [ 166.66666667]]


## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [226]:
for i in range(numElems):
    kMatrix = create_truss_k(angles.value[i])
    kMatrix *= barConstant[i]
    print(f"\nk matrix for element {i+1}: \n{kMatrix}")
    #print(kMatrix)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starting at node: {startNode} and ending at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, globalK, 0, 0, kMatrix)
    block_array_adjust(En, Sn, globalK, 2, 0, kMatrix)
    block_array_adjust(Sn, En, globalK, 0, 2, kMatrix)
    block_array_adjust(En, En, globalK, 2, 2, kMatrix)

print(f"\nGlobal K matrix: \n{globalK}")
unmodified_globalK = globalK.copy()
check_Symmetric(globalK)


k matrix for element 1: 
[[ 327.71961766  136.54984069 -327.71961766 -136.54984069]
 [ 136.54984069   56.89576695 -136.54984069  -56.89576695]
 [-327.71961766 -136.54984069  327.71961766  136.54984069]
 [-136.54984069  -56.89576695  136.54984069   56.89576695]]
Element 1 starting at node: 1 and ending at node: 2

k matrix for element 2: 
[[ 416.66666667    0.         -416.66666667   -0.        ]
 [   0.            0.           -0.           -0.        ]
 [-416.66666667   -0.          416.66666667    0.        ]
 [  -0.           -0.            0.            0.        ]]
Element 2 starting at node: 1 and ending at node: 3

k matrix for element 3: 
[[    0.    -0.    -0.     0.]
 [   -0.  1000.     0. -1000.]
 [   -0.     0.     0.    -0.]
 [    0. -1000.    -0.  1000.]]
Element 3 starting at node: 2 and ending at node: 3

k matrix for element 4: 
[[ 327.71961766  136.54984069 -327.71961766 -136.54984069]
 [ 136.54984069   56.89576695 -136.54984069  -56.89576695]
 [-327.71961766 -136.54

In [227]:
j = 0
while j < numNodes*2:
    print(f"{j}")
    #if j in nodesWithLoads:
     #   j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #globalK to reflect the changes.
    if uvList[j] == 0:
        print(f"u = 0 at row: {j}")
        forceList[j] = 0 
        globalK[j] = 0
        globalK[j,j] = 1
        j+=1
    elif uvList[j] == None:
        print(f"none detected at row {j}")
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print(f"row {j} altered")
        print(uvList[j])
        forceList[j] = uvList[j]
        for c in range(numNodes):
            if (globalK[j,c] > 0) == True:
                globalK[j,c] = 1
            else:
                globalK[j,c] = 0
        j+=1

                
print(globalK)
print(forceList)

0
none detected at row 0
1
none detected at row 1
2
none detected at row 2
3
none detected at row 3
4
u = 0 at row: 4
5
u = 0 at row: 5
6
none detected at row 6
7
none detected at row 7
8
none detected at row 8
9
none detected at row 9
10
u = 0 at row: 10
11
none detected at row 11
12
u = 0 at row: 12
13
none detected at row 13
[[  744.38628433   136.54984069  -327.71961766  -136.54984069
   -416.66666667     0.             0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [  136.54984069    56.89576695  -136.54984069   -56.89576695
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [ -327.71961766  -136.54984069   655.43923532   273.09968138
     -0.             0.          -327.71961766  -136.54984069
      0.             0.             0.             0.
      0.             0.        ]
 [ -136.54984069   -56.89576695   273.09968138  1113.

In [228]:
#globalK.round(0)
globalK_inv = inv(globalK)
#print(globalK_inv)

node = 1

uFinal = np.matmul(globalK_inv, forceList)

for i in range(len(uFinal)):
    if check_even(i) == True:
        print(f"u{node} = {uFinal[i]}")
    else:
        print(f"v{node} = {uFinal[i]}")
        node += 1 

u1 = [0.1152]
v1 = [-1.35939187]
u2 = [-0.30474661]
v2 = [0.]
u3 = [-0.]
v3 = [0.]
u4 = [0.00105693]
v4 = [-0.3824085]
u5 = [-0.0048]
v5 = [-0.4324085]
u6 = [-0.]
v6 = [-0.46775187]
u7 = [0.]
v7 = [-0.52775187]


Forces can be found from [K]*[u]

In [229]:
forcesFinal = np.matmul(unmodified_globalK, uFinal)
for i in range(len(forceList)):
    if forceList[i] != forcesFinal[i] and forceList[i] != 0 and forceList[i] != None:
        forcesFinal[i] = forceList[i]
print(f"Final Forces: \n{forcesFinal}")

node = 1

for i in range(len(forcesFinal)):
    if check_even(i) == True:
        print(f"fx{node} = {forcesFinal[i]}")
    else:
        print(f"fy{node} = {forcesFinal[i]}")
        node += 1 


Final Forces: 
[[ -0.]
 [-20.]
 [  0.]
 [  0.]
 [ 14.]
 [ 50.]
 [ -0.]
 [  0.]
 [  0.]
 [-20.]
 [-16.]
 [ -0.]
 [  2.]
 [-10.]]
fx1 = [-0.]
fy1 = [-20.]
fx2 = [0.]
fy2 = [0.]
fx3 = [14.]
fy3 = [50.]
fx4 = [-0.]
fy4 = [0.]
fx5 = [0.]
fy5 = [-20.]
fx6 = [-16.]
fy6 = [-0.]
fx7 = [2.]
fy7 = [-10.]


## Finding Stresses

In [230]:
stressFinal = np.zeros((numElems, 1))
#print(stressFinal)
for i in range(numElems):
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"\nElement {i+1} starts at node: {startNode} and ends at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    print(f"adjusted start node: {Sn} adjusted end node: {En}")
    trigMatrix = create_truss_stress_trig_matrix(angles.value[i])
    print(f"trigMatrix = \n{trigMatrix}")
    local_uMatrix = np.array([
        uFinal[Sn], 
        uFinal[(Sn + 1)], 
        uFinal[En], 
        uFinal[(En+1)]
        ])
    print(f"local u matrix: \n{local_uMatrix}")
    localLength = np.array([[-(1/lengths.value[i]), 1/lengths.value[i]]])
    print(f"local length matrix: \n{localLength}")

    indStress = (trigMatrix @ local_uMatrix)
    print(f"IndStress 1: {indStress}")
    indStress = localLength @ indStress
    indStress *= E.value

    stressFinal[i] = indStress
    
    
    print(f"Stress in element = {indStress}")

    


Element 1 starts at node: 1 and ends at node: 2
adjusted start node: 0 adjusted end node: 2
trigMatrix = 
[[0.92307692 0.38461538 0.         0.        ]
 [0.         0.         0.92307692 0.38461538]]
local u matrix: 
[[ 0.1152    ]
 [-1.35939187]
 [-0.30474661]
 [ 0.        ]]
local length matrix: 
[[-0.01282051  0.01282051]]
IndStress 1: [[-0.41650457]
 [-0.28130457]]
Stress in element = [[52.]]

Element 2 starts at node: 1 and ends at node: 3
adjusted start node: 0 adjusted end node: 4
trigMatrix = 
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]]
local u matrix: 
[[ 0.1152    ]
 [-1.35939187]
 [-0.        ]
 [ 0.        ]]
local length matrix: 
[[-0.01388889  0.01388889]]
IndStress 1: [[ 0.1152]
 [-0.    ]]
Stress in element = [[-48.]]

Element 3 starts at node: 2 and ends at node: 3
adjusted start node: 2 adjusted end node: 4
trigMatrix = 
[[ 0. -1.  0.  0.]
 [ 0.  0.  0. -1.]]
local u matrix: 
[[-0.30474661]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]]
local length matrix: 
[[-0.03333333  0.0333

In [231]:
print(stressFinal)

[[ 52.        ]
 [-48.        ]
 [  0.        ]
 [ 52.        ]
 [-78.10249676]
 [ -2.        ]
 [ 25.        ]
 [-13.        ]
 [ -6.40312424]
 [  2.        ]
 [ 20.        ]]


In [232]:
np.set_printoptions(suppress=True, formatter={'float':"{:0.4f}".format})
printTrussNodalDisplacements(uFinal, lengths.unit)
printStresses(stressFinal,E.unit)

████████████████████████████████████████████████████████████████████████████████████████████████████
The nodal displacements (u,v), in [in], ordered from lowest to highest numbered node are:
[[0.1152]
 [-1.3594]
 [-0.3047]
 [0.0000]
 [-0.0000]
 [0.0000]
 [0.0011]
 [-0.3824]
 [-0.0048]
 [-0.4324]
 [-0.0000]
 [-0.4678]
 [0.0000]
 [-0.5278]]
████████████████████████████████████████████████████████████████████████████████████████████████████
████████████████████████████████████████████████████████████████████████████████████████████████████

The element axial stresses, in [ksi], ordered from lowest to highest numbered element are:
[[52.0000]
 [-48.0000]
 [0.0000]
 [52.0000]
 [-78.1025]
 [-2.0000]
 [25.0000]
 [-13.0000]
 [-6.4031]
 [2.0000]
 [20.0000]]

Note: a (+) value indicates the element is in tension, while a (-) value in compression.

████████████████████████████████████████████████████████████████████████████████████████████████████
